# Import Libraries

In [ ]:
import importlib
import sys
sys.modules['imp'] = importlib

In [ ]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from IPython import display

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
params_cfg = {
    "action"   : "train_feat01",
    "feat_path": "/content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/data01.npz",
    "seed"    : 42, # Set random seed
    "exp_dir" : os.path.abspath('/content/drive/MyDrive/house_predict/process/exps'),
    'exp_name': 'trainbase_31102025',
    "data_dir": os.path.abspath("/content/drive/MyDrive/house_predict/data"),
    "verbose" : True,
}

params_cfg.update(**{
    "save_dir": os.path.abspath(f'{params_cfg["exp_dir"]}/{params_cfg["exp_name"]}')
})

for v in params_cfg:
    print(f'+ {v}: {params_cfg[v]}')

globals().update(**params_cfg)

+ action: train_feat01
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/trainbase_31102025/data01.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: trainbase_31102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/trainbase_31102025


# Data Load

In [ ]:
df_train = pd.read_csv(f'{data_dir}/train.csv')
df_test = pd.read_csv(f'{data_dir}/test.csv')

if params_cfg["verbose"]:
    print("-"*10, "information", "-"*10)
    print(f'train-col: {set(df_train.columns)}')
    print(f'test-col: {set(df_test.columns)}')
    print("Union:", set(df_train.columns).intersection(set(df_test.columns)))
    print("Difference:", set(df_train.columns).difference(set(df_test.columns)))

---------- information ----------
train-col: {'BsmtFinType2', 'GarageYrBlt', 'GarageCond', 'LandSlope', 'Functional', 'OverallQual', 'LowQualFinSF', 'YrSold', '2ndFlrSF', 'BsmtHalfBath', 'Heating', 'Utilities', 'MSSubClass', 'OpenPorchSF', 'TotalBsmtSF', 'PoolArea', 'BsmtFinSF2', 'TotRmsAbvGrd', 'ExterQual', '1stFlrSF', 'Exterior2nd', 'KitchenQual', 'Fence', 'YearRemodAdd', 'GarageCars', 'PoolQC', 'BsmtUnfSF', 'Street', 'LandContour', 'BsmtFinType1', 'BsmtExposure', 'LotFrontage', 'FireplaceQu', 'LotShape', 'BldgType', 'SaleType', 'GarageArea', 'MiscFeature', 'PavedDrive', 'YearBuilt', 'GrLivArea', 'SaleCondition', 'BsmtFullBath', 'RoofMatl', 'GarageType', 'LotConfig', 'HalfBath', 'ScreenPorch', 'Id', 'BsmtFinSF1', 'Foundation', '3SsnPorch', 'Electrical', 'MiscVal', 'SalePrice', 'OverallCond', 'ExterCond', 'HouseStyle', 'CentralAir', 'BedroomAbvGr', 'HeatingQC', 'Condition1', 'KitchenAbvGr', 'BsmtQual', 'MSZoning', 'FullBath', 'Alley', 'Fireplaces', 'RoofStyle', 'BsmtCond', 'WoodDeckSF

# Processing

In [ ]:
df_output = pd.DataFrame()
df_output = df_train.copy()

In [ ]:
def preprocessing_feature_01(df_data, is_train=True, is_debug=True, **kwargs):
    df_output = df_data.copy()


    # Xóa những cột giá trị thiếu trên 50%
    missing_ratio = df_output.isnull().sum() / len(df_output)
    cols_to_drop = missing_ratio[missing_ratio >= 0.5].index
    df_output = df_output.drop(cols_to_drop, axis=1)

    df_output = df_output.drop('FireplaceQu', axis=1, errors='ignore')

    # ---- 1️⃣ Điền giá trị thiếu cho dạng số ----
    num_cols = df_output.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = df_output.select_dtypes(include=['object','category']).columns
    for col in num_cols:
        if df_output[col].isnull().sum() > 0:
            median_value = df_output[col].median()
            df_output[col] = df_output[col].fillna(median_value)

    # ---- 2️⃣ Điền giá trị thiếu cho dạng category ----
    cols_fill_none = [
        'GarageFinish', 'GarageType', 'GarageQual', 'GarageCond',
        'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual',
    ]
    for col in cols_fill_none:
        if col in df_output.columns:
            df_output[col] = df_output[col].fillna('None')


    df_output['Electrical'] = df_output['Electrical'].fillna(df_output['Electrical'].mode()[0])

    cols_fill_mode = [
    'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
    'KitchenQual', 'Functional', 'SaleType'
    ]

    for col in cols_fill_mode:
        if col in df_output.columns:
            df_output[col] = df_output[col].fillna(df_output[col].mode()[0])


    # ---- 3️⃣ Feature engineering ----
    # === FEATURE ENGINEERING ===
    # Kết hợp, xử lý và tạo feature mới giúp mô hình học tốt hơn

    # --- Tổng diện tích sử dụng ---
    if all(c in df_output.columns for c in ['GrLivArea', 'TotalBsmtSF', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']):
        df_output['TotalArea'] = (
            df_output['GrLivArea'] + df_output['TotalBsmtSF'] +
            df_output['GarageArea'] + df_output['WoodDeckSF'] + df_output['OpenPorchSF']
        )

    # --- Chất lượng ngoại thất + tổng thể ---
    if all(c in df_output.columns for c in ['ExterQual', 'OverallQual']):
        df_output['Qual_Exter_Overall'] = (
            df_output['ExterQual'] + '_' + df_output['OverallQual'].astype(str)
        )

    # --- Tuổi sau khi xây & sau khi cải tạo ---
    if all(c in df_output.columns for c in ['YrSold', 'YearBuilt']):
        df_output['AgeSinceBuilt'] = df_output['YrSold'] - df_output['YearBuilt']

    if all(c in df_output.columns for c in ['YrSold', 'YearRemodAdd']):
        df_output['RemodAge'] = df_output['YrSold'] - df_output['YearRemodAdd']

    if all(c in df_output.columns for c in ['YearBuilt', 'YearRemodAdd']):
        df_output['YearsBeforeRemod'] = df_output['YearRemodAdd'] - df_output['YearBuilt']

    # --- Tổng số phòng tắm (bao gồm tầng hầm) ---
    bath_cols = ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']
    if all(c in df_output.columns for c in bath_cols):
        df_output['Baths_Total'] = (
            df_output['FullBath'] + 0.5 * df_output['HalfBath'] +
            df_output['BsmtFullBath'] + 0.5 * df_output['BsmtHalfBath']
        )

    # --- Tổng điểm chất lượng x diện tích ---
    if all(c in df_output.columns for c in ['OverallQual', 'LotArea']):
        df_output['Qual_LotArea'] = df_output['OverallQual'] * df_output['LotArea']

    if all(c in df_output.columns for c in ['TotalArea', 'OverallQual']):
        df_output['TotalArea_Qual'] = df_output['TotalArea'] * df_output['OverallQual']

    # --- Tỷ lệ diện tích sống / tổng số phòng ---
    if all(c in df_output.columns for c in ['GrLivArea', 'TotRmsAbvGrd']):
        df_output['AreaPerRoom'] = df_output['GrLivArea'] / (df_output['TotRmsAbvGrd'] + 1)

    # --- Kết hợp vị trí + loại nhà + kiểu nhà + điều kiện bán ---
    if all(c in df_output.columns for c in ['Neighborhood', 'BldgType', 'HouseStyle', 'SaleCondition']):
        df_output['Loc_Type_Style_Cond'] = (
            df_output['Neighborhood'] + '_' +
            df_output['BldgType'] + '_' +
            df_output['HouseStyle'] + '_' +
            df_output['SaleCondition']
        )

    # --- Sử dụng garage hợp lý ---
    if all(c in df_output.columns for c in ['GarageArea', 'GarageCars']):
        df_output['GarageUtilization'] = df_output['GarageArea'] / (df_output['GarageCars'] + 1)

    # --- Điểm tiện nghi (Amenity Score) ---
    if all(c in df_output.columns for c in ['Fireplaces', 'FullBath']):
        df_output['AmenityScore'] = df_output['Fireplaces'] * 2 + df_output['FullBath'] * 3

    # --- Tỷ lệ diện tích sống / diện tích đất ---
    if all(c in df_output.columns for c in ['GrLivArea', 'LotArea']):
        df_output['Living_to_LotRatio'] = df_output['GrLivArea'] / (df_output['LotArea'] + 1)

    # --- Chênh lệch chất lượng và điều kiện tổng thể ---
    if all(c in df_output.columns for c in ['OverallQual', 'OverallCond']):
        df_output['QualMinusCond'] = df_output['OverallQual'] - df_output['OverallCond']

    # --- Diện tích có thể sử dụng (bao gồm tầng hầm + garage) ---
    if all(c in df_output.columns for c in ['TotalBsmtSF', 'GrLivArea', 'GarageArea']):
        df_output['LivableArea'] = (
            df_output['TotalBsmtSF'] + df_output['GrLivArea'] + df_output['GarageArea']
        )

    # --- Tỷ lệ tầng hầm / tầng sống ---
    if all(c in df_output.columns for c in ['TotalBsmtSF', 'GrLivArea']):
        df_output['Bsmt_to_LivArea'] = df_output['TotalBsmtSF'] / (df_output['GrLivArea'] + 1)

    # --- Kết hợp vị trí và diện tích ---
    if all(c in df_output.columns for c in ['Neighborhood', 'LotArea']):
        df_output['Loc_Lot'] = (
            df_output['Neighborhood'] + '_' +
            pd.cut(df_output['LotArea'], bins=5, labels=False).astype(str)
        )

    # --- Thời gian bán sau khi cải tạo ---
    if all(c in df_output.columns for c in ['YrSold', 'YearRemodAdd']):
        df_output['SoldSinceRemod'] = df_output['YrSold'] - df_output['YearRemodAdd']





    # ---- 5️⃣ Debug ----
    if is_debug:
        print("✅ Hoàn tất preprocessing_feature_01")
        print(f"🔹 Dữ liệu đầu ra: {df_output.shape[0]} hàng, {df_output.shape[1]} cột")
        new_cols = [col for col in df_output.columns if col not in df_data.columns]
        print(f"✨ Số lượng feature mới: {len(new_cols)}")
        print("🆕 Feature mới:", new_cols[:15], "..." if len(new_cols) > 15 else "")

    return df_output, None


In [ ]:
def main_feat01(**kwargs):
  # load data
  df_train = pd.read_csv(f'{data_dir}/train.csv')
  df_test = pd.read_csv(f'{data_dir}/test.csv')
  # preprocessing
  df_output_train, _ = preprocessing_feature_01(df_train, is_train=True, is_debug=False)
  df_output_test, _ = preprocessing_feature_01(df_test, is_train=False, is_debug=False)

  # saving
  os.makedirs(save_dir, exist_ok=True)

  # Lưu trực tiếp DataFrame objects (cần allow_pickle=True khi load)
  np.savez(f'{save_dir}/data01.npz',
             train_data=df_output_train.values,
             test_data=df_output_test.values,
             train_columns=df_output_train.columns.values,  # Lưu tên cột train
             test_columns=df_output_test.columns.values,
             allow_pickle=True)

  print("Đã lưu DataFrame 01 với đầy đủ thông tin cột")

  kwargs.get('global_cfg', {}).update(**locals())

if params_cfg["action"] == "train_feat01":
    print("Runing ... [train_feat01]")
    main_feat01(global_cfg = globals())

Runing ... [train_feat01]
Đã lưu DataFrame 01 với đầy đủ thông tin cột
